<a href="https://colab.research.google.com/github/anastaszi/257_machine_learning/blob/master/HW10_Anastasia_Zimina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Imports

In [ ]:
!pip install --upgrade gensim
!pip install pyspellchecker 
!pip install contractions

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import wordcloud
import nltk
import unicodedata
import contractions
import pickle
%matplotlib inline

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from pandas.api.types import CategoricalDtype

from gensim.utils import simple_preprocess
from collections import defaultdict
from contractions import contractions_dict
from spellchecker import SpellChecker
from wordcloud import WordCloud, STOPWORDS

from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize.toktok import ToktokTokenizer

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')

from contractions import contractions_dict
from scipy import stats
from scipy import sparse

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
scaler = MinMaxScaler()
std_scaler = StandardScaler()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Part I

1. data ingestion and data prep : get  your 3 datasets
2. train your microfactors/factor:  based on the latest datasets you have scraped or used an api
3. save the trained model
4. factor inference : run predictions on your factor and store the results of the inference
5. classify the new data you brought in between 1-6 (true-o-meter ) classification based on politifact labels
6. compare results

## Datasets



*   LiarLiarDataset [Article](https://arxiv.org/abs/1705.00648) [Repo](https://sites.cs.ucsb.edu/~william/data/)
*   [Kaggle Fake News Corpus](https://www.kaggle.com/c/fake-news/data?select=train.csv)
*   GossipCop [FakeNewsNetRepo](https://github.com/KaiDMML/FakeNewsNet)



### Dataset LiarLiar


In [ ]:
df_ll = pd.read_csv('/content/drive/MyDrive/ML/AlternusVera-Datasets/LiarLiar_datasets/liarliartrain.tsv', delimiter='\t', dtype=object)

In [ ]:
df_ll.columns =['id', 'label', 'statement', 'subjects', 'speaker', 'speaker_job_title', 'state_info',  'party_affiliation', 'count_1', 'count_2','count_3','count_4','count_5','context' ]

In [ ]:
df_ll.head(1)

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affiliation,count_1,count_2,count_3,count_4,count_5,context
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.


In [ ]:
df_ll['label'].unique()

array(['half-true', 'mostly-true', 'false', 'true', 'barely-true',
       'pants-fire'], dtype=object)

In [ ]:
def conver_to_spam_metric(x):
  label = x['label']
  if (label == 'mostly-true' or label == 'true' or label == 'half-true'):
    return 0
  return 1

In [ ]:
df_ll['spam_label'] = df_ll.apply(conver_to_spam_metric, axis=1)

In [ ]:
df_ll.head(1)

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affiliation,count_1,count_2,count_3,count_4,count_5,context,spam_label
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.,0


In [ ]:
df_ll_new = df_ll[['statement', 'spam_label']]

In [ ]:
df_ll_new.head(1)

,statement,spam_label
0,When did the decline of coal start? It started...,0


### Dataset FakeNews


In [ ]:
df_fn = pd.read_csv('/content/drive/MyDrive/ML/AlternusVera-Datasets/Fake_news_kaggle_datasets/train.csv')

In [ ]:
df_fn.head(1)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1


In [ ]:
df_fn = df_fn[['title', 'label']]

In [ ]:
df_fn.rename(columns={'title': 'statement', 'label': 'spam_label'}, inplace=True)

In [ ]:
df_fn.head(1)

,statement,spam_label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1


### Dataset: GossipCap

Fake news from GossipCap

In [ ]:
df_gc_f = pd.read_csv('/content/drive/MyDrive/ML/AlternusVera-Datasets/GossipCopDataset/master_dataset_gossipcop_fake.csv')

In [ ]:
df_gc_f.head(1)

,id,news_url,title,tweet_ids
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...


In [ ]:
df_gc_f['spam_label'] = 1

In [ ]:
df_gc_f.drop(['id', 'news_url', 'tweet_ids'], axis=1, inplace=True)

In [ ]:
df_gc_f.head(1)

,title,spam_label
0,Did Miley Cyrus and Liam Hemsworth secretly ge...,1


In [ ]:
df_gc_f.rename(columns={'title': 'statement'}, inplace=True)

In [ ]:
df_gc_t = pd.read_csv('/content/drive/MyDrive/ML/AlternusVera-Datasets/GossipCopDataset/master_dataset_gossipcop_real.csv')

In [ ]:
df_gc_t['spam_label'] = 0

In [ ]:
df_gc_t.drop(['id', 'news_url', 'tweet_ids'], axis=1, inplace=True)

In [ ]:
df_gc_t.rename(columns={'title': 'statement'}, inplace=True)

In [ ]:
df_gc_t.head(1)

,statement,spam_label
0,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,0


### Combine all datasets together

In [ ]:
df = df_ll_new.copy()

In [ ]:
df = df.append(df_fn, ignore_index=False)

In [ ]:
df = df.append(df_gc_f, ignore_index=False)

In [ ]:
df = df.append(df_gc_t, ignore_index=False)

In [ ]:
df.shape

(53179, 2)

In [ ]:
df.to_csv('/content/drive/MyDrive/ML/AlternusVera-Datasets/spam_dataset_full.csv', index=False)

## Data Prep

In [ ]:
df[df.isna().any(axis=1)]

,statement,spam_label
53,NaN,1
120,NaN,1
124,NaN,1
140,NaN,1
196,NaN,1
...,...,...
20568,NaN,1
20627,NaN,1
20636,NaN,1
20771,NaN,1


In [ ]:
df = df.dropna();

In [ ]:
  def process_text(text):
    def remove_accented_chars(text):
      text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
      return text

    def remove_stopwords(words, stopwords):
      words = [w for w in  words  if not w in stopwords] 
      return words
    def remove_smallwords(words):
      words = [w for w in  words  if len(w)> 2] 
      return words

    doc = remove_accented_chars(text)
    # remove contractions
    doc = contractions.fix(doc)
    doc = doc.lower()
    # remove new lines
    doc = re.sub(r'[\r|\n|\r\n]+','', doc)
    # remove extra white spaces
    doc = re.sub(' +', ' ', doc)
    # remove special characters
    special_char_pattern = re.compile(r'([{.(-)!}])')
    # leave only words
    doc = re.sub('\W+',' ', doc)
    doc = re.sub(r'[^a-zA-Z\s]', '', doc)
    words = tokenizer.tokenize(doc)
    words = [token.strip() for token in words]
    # lemmatize
    words = [ WordNetLemmatizer().lemmatize(w) for w in words ]
    words = remove_stopwords(words, stopword_list)
    words = remove_smallwords(words)
    return words

Tokenize statements into words:

In [ ]:
df['words'] = df['statement'].apply(process_text)

In [ ]:
def POS_count(words):
  tagged_words = nltk.pos_tag(words, tagset='universal');
  num_words = len(tagged_words)
  fq= defaultdict( int )
  for (word, pos) in tagged_words:
    fq[pos] += 1
  # share of nouns etc...
  if num_words == 0:
    return 0,0,0,0,0,0,0
  nouns = fq['NOUN']/num_words
  adjs = fq['ADJ'] /num_words
  prep = (fq['ADP'] + fq['CONJ'])/num_words
  dets = fq['DET']/num_words
  vbs = fq['VERB']/num_words
  advs = fq['ADV']/num_words
  prons = fq['PRON']/num_words
  return nouns, adjs, prep, dets, vbs, advs, prons

- 'noun' = Noun,
- 'adj' = Adjective,
- 'prep' = Adposition and conjunction,
- 'dt' = Determiner,
- 'verb' = Verb,
- 'adv' = Adverb,
- 'pron' = Pronoun

In [ ]:
df['noun'], df['adj'], df['prep'], df['dt'], df['verb'], df['adv'], df['pron'] = zip(*df['words'].map(POS_count))

In [ ]:
spell = SpellChecker()
def count_misspelled(words):
  misspelled = spell.unknown(words)
  return len(misspelled)

In [ ]:
df['count_mis'] = df.apply(lambda x: count_misspelled(x['words']), axis=1)

In [ ]:
df.to_csv("/content/drive/MyDrive/ML/AlternusVera-Datasets/spam_dataset_processed_full.csv', index=False", sep=',')

In [ ]:
X = df.drop('spam_label', axis=1)
y = df['spam_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
tfidf = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x)
X_train_tfidf_features = tfidf.fit_transform(X_train['words'])
X_test_tfidf_features = tfidf.transform(X_test['words'])

In [ ]:
tf_transformer_vocab = tfidf.vocabulary_

In [ ]:
print('TFIDF model:> Train features shape:', X_train_tfidf_features.shape, 'TFIDF model:> Test features shape:', X_test_tfidf_features.shape)

TFIDF model:> Train features shape: (35256, 23918) TFIDF model:> Test features shape: (17365, 23918)


## Model Training Spam Score

In [ ]:
classifiers = []
classifiers.append(('Multinomial NB', MultinomialNB(alpha=1)))
# classifiers.append(("Logistic Regression", LogisticRegression(random_state=42)))
# classifiers.append(("RandomForestClassifier", RandomForestClassifier(random_state=42)))

results = []

In [ ]:
def model_loop(models, X_train, X_test, y_train, y_test, factor, results):
  trained_models=[]
  for (name, model) in models:
    model.fit(X_train, y_train)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)
    cv_mean_score = np.mean(cv_scores)
    test_score = model.score(X_test, y_test)
    results.append([factor, name, cv_scores, cv_mean_score, test_score])
    trained_models.append((name, model))
  return trained_models

In [ ]:
pos_columns = ["noun","adj","prep",	"dt", "verb",	"adv"]
X_train_pos_features = scaler.fit_transform(X_train[pos_columns])
X_test_pos_features = scaler.fit_transform(X_test[pos_columns])

In [ ]:
X_train_spell_features = scaler.fit_transform(X_train[['count_mis']])
X_test_spell_features = scaler.fit_transform(X_test[['count_mis']])

In [ ]:
X_train_spam = sparse.hstack([X_train_tfidf_features, X_train_spell_features, X_train_pos_features]).tocsr()
X_test_spam  = sparse.hstack([X_test_tfidf_features, X_test_spell_features, X_test_pos_features]).tocsr()

In [ ]:
spam_models = model_loop(classifiers, X_train_spam, X_test_spam, y_train, y_test, 'SPAM', results);

In [ ]:
df_results = pd.DataFrame(results, columns=['Factor', 'Model', 'CV Accuracy (5-fold)', 'Mean CV Accuracy', 'Test Accuracy'])
df_results

,Factor,Model,CV Accuracy (5-fold),Mean CV Accuracy,Test Accuracy
0,SPAM,Multinomial NB,"[0.7492909812819059, 0.7519500780031201, 0.750...",0.751135,0.755428


In [ ]:
pickle.dump(tfidf.vocabulary_, open("/content/drive/MyDrive/ML/FEATURE_MODELS/tfidf_vocab.pkl", "wb"))

In [ ]:
pickle.dump(spam_models[0][1], open("/content/drive/MyDrive/ML/FEATURE_MODELS/spam_model.pkl", "wb"))

## Model Training Classification

> Indented block



### Politifact dataset


In [ ]:
df_poli = pd.read_csv('/content/drive/MyDrive/ML/AlternusVera-Datasets/scraped_data.csv')

In [ ]:
df_poli.head(1)

,author,statement,source,date,target
0,Kiannah Sepeda-Miller,"The migrant caravan from Tapachula, Mexico “is...",Mary Miller,"October 31, 2021",pants-fire


In [ ]:
df_poli.groupby('target').count()

,author,statement,source,date
target,,,,
barely-true,346,346,346,346
false,1165,1165,1165,1165
full-flop,13,13,13,13
half-flip,4,4,4,4
half-true,223,223,223,223
mostly-true,137,137,137,137
no-flip,1,1,1,1
pants-fire,417,417,417,417
true,94,94,94,94


In [ ]:
df_poli['target'].unique()

array(['pants-fire', 'false', 'barely-true', 'mostly-true', 'true',
       'half-true', 'half-flip', 'full-flop', 'no-flip'], dtype=object)

In [ ]:
df_poli.drop(df_poli[df_poli['target'] == 'half-flip'].index, inplace = True)
df_poli.drop(df_poli[df_poli['target'] == 'full-flop'].index, inplace = True)
df_poli.drop(df_poli[df_poli['target'] == 'no-flip'].index, inplace = True)

In [ ]:
df_poli.shape

(2382, 5)

In [ ]:
cat_type = CategoricalDtype(categories=["true", "mostly-true", "half-true", 'barely-true', 'false', 'pants-fire'], ordered=True)

In [ ]:
df_poli['label'] = df_poli['target'].astype(cat_type)

In [ ]:
df_poli.drop(['author', 'source', 'date', 'target'], axis=1,inplace=True)

In [ ]:
df_poli.head(1)

,statement,label
0,"The migrant caravan from Tapachula, Mexico “is...",pants-fire


### Predict spam score for politifact dataset

In [ ]:
tf_vocab = pickle.load(open("/content/drive/MyDrive/ML/FEATURE_MODELS/tfidf_vocab.pkl", 'rb'))
spam_model = pickle.load(open("/content/drive/MyDrive/ML/FEATURE_MODELS/spam_model.pkl", 'rb'))

In [ ]:
class Spam():

  def POS_count(self, words):
    tagged_words = nltk.pos_tag(words, tagset='universal');
    num_words = len(tagged_words)
    fq= defaultdict( int )
    for (word, pos) in tagged_words:
      fq[pos] += 1
    # share of nouns etc...
    if num_words == 0:
      return 0,0,0,0,0,0,0
    nouns = fq['NOUN']/num_words
    adjs = fq['ADJ'] /num_words
    prep = (fq['ADP'] + fq['CONJ'])/num_words
    dets = fq['DET']/num_words
    vbs = fq['VERB']/num_words
    advs = fq['ADV']/num_words
    prons = fq['PRON']/num_words
    return nouns, adjs, prep, dets, vbs, advs, prons
  def remove_accented_chars(self, text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text
  
  def remove_stopwords(self, words, stopwords):
    words = [w for w in  words  if not w in stopwords] 
    return words

  def remove_smallwords(self,words):
    words = [w for w in  words  if len(w)> 2] 
    return words

  def process_text(self, text):
    custom_stopwords = self.custom_stopwords
    doc = self.remove_accented_chars(text)
    # remove contractions
    doc = contractions.fix(doc)
    doc = doc.lower()
    # remove new lines
    doc = re.sub(r'[\r|\n|\r\n]+','', doc)
    # remove extra white spaces
    doc = re.sub(' +', ' ', doc)
    # remove special characters
    special_char_pattern = re.compile(r'([{.(-)!}])')
    # leave only words
    doc = re.sub('\W+',' ', doc)
    doc = re.sub(r'[^a-zA-Z\s]', '', doc)
    words = tokenizer.tokenize(doc)
    words = [token.strip() for token in words]
    # lemmatize
    words = [ WordNetLemmatizer().lemmatize(w) for w in words ]
    words = self.remove_stopwords(words, custom_stopwords)
    words = self.remove_smallwords(words)
    return words

  def __init__(self, model, tf_vocab_):
    self.model = model
    self.custom_stopwords = stopword_list + ['wa', 'mr']
    self.sc = MinMaxScaler()
    self.pos_columns = ["noun","adj","prep",	"dt", "verb",	"adv"]
    self.spell = SpellChecker()
    self.tfidf = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x,analyzer='word', vocabulary = tf_vocab_)
  
  def count_misspelled(self,words):
    misspelled = self.spell.unknown(words)
    return len(misspelled)
  
  def create_df(self, text):
    dfrme = pd.DataFrame(index=[0], columns=['text'])
    dfrme['text'] = text
    dfrme['words'] = dfrme['text'].apply(self.process_text)
    return dfrme
  
  def predict_spam(self, text):
    dfrme = self.create_df(text)
    dfrme['noun'], dfrme['adj'], dfrme['prep'], dfrme['dt'], dfrme['verb'], dfrme['adv'], dfrme['pron'] = zip(*dfrme['words'].map(self.POS_count))
    dfrme['count_mis'] = dfrme.apply(lambda x: self.count_misspelled(x['words']), axis=1)
    spell_features = self.sc.fit_transform(dfrme[['count_mis']])
    pos_features = self.sc.fit_transform(dfrme[self.pos_columns])
    tfidf_features = self.tfidf.fit_transform(dfrme['words'])
    combined_features = sparse.hstack([spell_features, pos_features, tfidf_features]).tocsr()
    prediced = self.model.predict(combined_features)
    predicedProb = self.model.predict_proba(combined_features)
    return (prediced[0], predicedProb[0][1])

In [ ]:
spam = Spam(spam_model, tf_vocab)
def CLIMATEFRIENDS_getSpamScore(text):  # return between 0 and 1, being 0 = True,  1 = Fake
    binaryValue, probValue = spam.predict_spam(text)
    return probValue

In [ ]:
df_poli['spam_score'] = df_poli['statement'].apply(CLIMATEFRIENDS_getSpamScore)

In [ ]:
df_poli.to_csv("/content/drive/MyDrive/ML/AlternusVera-Datasets/spam_politifact_dataset_processed_full.csv', index=False", sep=',')

In [ ]:
poli_classifiers = []
poli_classifiers.append(("Logistic Regression", LogisticRegression(random_state=42)))
poli_classifiers.append(("RandomForestClassifier", RandomForestClassifier(random_state=42)))

final_results = []

In [ ]:
X_poli = df_poli.drop(['label', 'statement'], axis=1)
y_poli = df_poli['label']
X_poli_train, X_poli_test, y_poli_train, y_poli_test = train_test_split(X_poli, y_poli, test_size=0.33, random_state=42)

In [ ]:
poli_models = model_loop(poli_classifiers, X_poli_train, X_poli_test, y_poli_train, y_poli_test, 'Poli_classifier', final_results);

In [ ]:
df_poli_results = pd.DataFrame(final_results, columns=['Factor', 'Model', 'CV Accuracy (5-fold)', 'Mean CV Accuracy', 'Test Accuracy'])
df_poli_results

,Factor,Model,CV Accuracy (5-fold),Mean CV Accuracy,Test Accuracy
0,Poli_classifier,Logistic Regression,"[0.4952978056426332, 0.4952978056426332, 0.492...",0.493417,0.480305
1,Poli_classifier,RandomForestClassifier,"[0.30094043887147337, 0.3072100313479624, 0.23...",0.287774,0.320203


In [ ]:
pickle.dump(poli_models[0], open("/content/drive/MyDrive/ML/FEATURE_MODELS/poli_spam_classif_model.pkl", "wb"))

In [ ]:
text = df_poli[df_poli.label == 'true']['statement'].to_numpy()[0]

In [ ]:
print(text)

The Biden administration "published a study concluding 4 (of) 5 new cars on the road by 2050 will still require liquid fuels."


In [ ]:
class PoliSpamClassif():
  def __init__(self, model_spam_score, vocab, model_classif):
    self.model_classif = model_classif
    self.spam = Spam(model_spam_score, vocab)

  def predict(self,text):
    binaryValue, probValue = spam.predict_spam(text)
    dfrme = pd.DataFrame(index=[0], columns=['spam_score'])
    dfrme['spam_score'] = probValue
    prediced = self.model_classif.predict(dfrme)
    predicedProb = self.model_classif.predict_proba(dfrme)
    print(predicedProb)
    return prediced

In [ ]:
polispam = PoliSpamClassif(spam_model, tf_vocab, poli_models[0][1])

In [ ]:
polispam.predict("Migrants are crossing the border")

[[0.144218   0.48549541 0.09073293 0.05958213 0.17953905 0.04043247]]


array(['false'], dtype=object)

# Part II

team work in [this notebook](https://colab.research.google.com/drive/1u5lL8hOxaHcLR-j01CrKxpNCQaHbutcq#scrollTo=ZTWGIaNiCkzI)